<a href="https://colab.research.google.com/github/essexgroup31/scrappy-doo/blob/main/Copy_of_Covid19_Twitter_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Tweepy
#!pip install tweepy
# Install Matplotlib
#!pip install matplotlib


In [3]:
def removeSuffix(item,suffix):
  if item.endswith(suffix):
    return item[:-len(suffix)]
  else:
    return item[:]

In [15]:
import string
def punctuationRemover(sentence):
    myList = []
    for each in range(len(sentence)-1):
        if sentence[each] in string.punctuation:
            if sentence[each+1] in string.ascii_letters:
                myList.append(sentence[each])
                myList.append(" ")
            else:
                myList.append(sentence[each])
        else:
            myList.append(sentence[each])   # <------------------ HERE IS THE ERROR
                
    
    for each in range(len(myList)):
        if myList[each] in string.punctuation:
            myList[each] = "/"

    sentence = ""

    for each in myList:
        if each != "/":
          sentence = sentence + str(each)

    return sentence

In [5]:
def stripShire(sentence):
    sentence = sentence.split()
    completedsentence = []
    
    for each in sentence:
        completedsentence.append(removeSuffix(each, "shire"))
    

    return completedsentence

In [6]:
import re
import tweepy
from tweepy import OAuthHandler
consumer_api_key = 'uQP9N32xpuEIujXsDV2VgcekC'
consumer_api_secret = 'aLWDUV2P718ZoOicHCgpvQ8JUWgaOVcce14iA5oXGdtb0PsEV8' 
access_token = '1315962347836579840-WKXz7fp8UEZAaOtABYv1fIpoZpXX7q'
access_token_secret ='mitdeExvQ0SFuPGn3HsUhG0faElwQQPSE7FmhaZBNvh76'
print (consumer_api_key)
print (access_token)

uQP9N32xpuEIujXsDV2VgcekC
1315962347836579840-WKXz7fp8UEZAaOtABYv1fIpoZpXX7q


In [7]:
authorizer = OAuthHandler(consumer_api_key, consumer_api_secret)
authorizer.set_access_token(access_token, access_token_secret)

In [8]:
#search_query = input("Hello user, what would you like to search?\nSearch: ")

In [10]:
api = tweepy.API(authorizer ,timeout=15, wait_on_rate_limit=True)
all_tweets = []
ukTweetList = []
search_query = 'covid19'

for tweet_object in tweepy.Cursor(api.search,q=search_query+" -filter:retweets",lang='en',result_type='recent').items(200):
    all_tweets.append(tweet_object.text)
    print(tweet_object.author.location)
    if ("UK" or "United Kingdom") in tweet_object.user.location:
      ukTweetList.append(tweet_object)
    

In [16]:
ukCounties = ["Bedford","Berk","Bristol","Buckingham","Cambridge","Ches","Cornwall","Cumbria","Derby","Devon","Dorset","Durham","Sussex","Essex","Gloucester","City of London", "Manchester", "Hamp","Hereford","Hertford","Isle of Wight", "Kent", "Lanca", "Leicester","Lincoln","London", "Merseyside", "Norfolk", "Northampton", "Northumberland", "North York", "Nottingham", "Oxford", "Rutland", "Shrop", "Somerset","South York", "Stafford", "Suffold", "Surrey", "Tyne and Wear", "Warwick", "West Midlands", "West Sussex", "West York", "Wilt", "Worcester"]
tweetLocationData = []
# List of uk counties without "shire" at the end (because of the way that twitter works with locations)
for each in ukTweetList:
    tweetData = each
    location = each.user.location
    
    #time to die :D (kill me)
    location = punctuationRemover(location)
    location = stripShire(location)

    tweetData = "d"

    for x in location:
        if x in ukCounties:
          location = x
          if len(tweetLocationData) == 0:
            tweetLocationData.append([location,tweetData])
            
          else:
            for each in range(len(tweetLocationData)):
              if tweetLocationData[each][0] == location:
                tweetLocationData[each].append(tweetData)
                break
              else:
                tweetLocationData.append([location, tweetData])
                break
            

print(tweetLocationData)
    
    



West Midlands, UK
['West', 'Midlands', 'U']
Worldwide - offices in UK+Aus
['Worldwide', 'offices', 'in', 'UK', 'Au']
London, UK
['London', 'U']
Canada | USA | UK
['Canada', 'USA', 'U']
London, UK
['London', 'U']
UK
['U']
[['London', 'd', 'd']]


In [ ]:
import numpy as np 
import pandas as pd 
import re  
import nltk # an amazing library to play with natural language
nltk.download('stopwords')
from nltk.corpus import stopwords


In [ ]:
tweets = pd.read_csv("https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv")
print(tweets)

In [ ]:
X = tweets.iloc[:, 10].values
y = tweets.iloc[:, 1].values

print(X)

In [ ]:
processed_tweets = []
 
for tweet in range(0, len(X)):  
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', str(X[tweet]))
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    processed_tweets.append(processed_tweet)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(processed_tweets).toarray()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=1)  
text_classifier.fit(X, y)

In [ ]:
negativeCounter = 0
neutralCounter = 0
positiveCounter = 0
for tweet in all_tweets:
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', tweet)
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    sentiment = text_classifier.predict(tfidfconverter.transform([ processed_tweet]).toarray())
    

    # We need to create something that counts the different sentiments
    # Which will then be the data for the pie chart.

    if sentiment == "positive":
      positiveCounter+=1
    elif sentiment == "negative":
      negativeCounter+=1
    elif sentiment == "neutral":
      neutralCounter+=1
   
    #print(tweet, sentiment)


    

#print(all_tweets)
print("Positive sentiments:",positiveCounter)
print("Negative sentiments:",negativeCounter)
print("Neutral sentiments:",neutralCounter)

In [ ]:
import matplotlib.pyplot as pyplot
labels = "Negative", "Neutral", "Positive"
sizes = [negativeCounter,neutralCounter,positiveCounter]
print(sizes)
colors = ["red","yellow","green"]

if sizes[0] > sizes[1] and sizes[0] > sizes[1]:
    explode = (0.1,0,0)
elif sizes[1] > sizes[0] and sizes[1] > sizes[2]:
    explode = (0,0.1,0)
else:
    explode = (0,0,0.1)


pyplot.pie(sizes,explode = explode, labels=labels,colors=colors,autopct="%1.1f%%", shadow=True, startangle=140)
pyplot.axis("equal")
pyplot.show()
